In [1]:
import ipyvolume as ipv
import numpy as np

s = 1/2**0.5
# 4 vertices for the tetrahedron
x = np.array([1.,  -1, 0,  0])
y = np.array([0,   0, 1., -1])
z = np.array([-s, -s, s,  s])
# and 4 surfaces (triangles), where the number refer to the vertex index
triangles = [(0, 1, 2), (0, 1, 3), (0, 2, 3), (1,3,2)]

ipv.figure()
# we draw the tetrahedron
mesh = ipv.plot_trisurf(x, y, z, triangles=triangles, color='orange')
# and also mark the vertices
ipv.scatter(x, y, z, marker='sphere', color='blue')
ipv.xyzlim(-2, 2)
ipv.show()

In [2]:
import ipyvolume as ipv
import numpy as np

# f(u, v) -> (u, v, u*v**2)
a = np.arange(-5, 5)
U, V = np.meshgrid(a, a)
X = U
Y = V
Z = X*Y**2

ipv.figure()
ipv.plot_surface(X, Z, Y, color="orange")
ipv.plot_wireframe(X, Z, Y, color="red")
ipv.show()

In [3]:
import ipyvolume as ipv
import numpy as np

X = np.arange(-5, 5, 0.25*1)
Y = np.arange(-5, 5, 0.25*1)
X, Y = np.meshgrid(X, Y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)

from matplotlib import cm
colormap = cm.coolwarm
znorm = Z - Z.min()
znorm /= znorm.ptp()
znorm.min(), znorm.max()
color = colormap(znorm)

ipv.figure()
mesh = ipv.plot_surface(X, Z, Y, color=color[...,:3])
ipv.show()

In [4]:
import numpy as np
from numpy import pi, sin, cos, radians
import ipyvolume as ipv
import ipywidgets as widgets

# only x is a sequence of arrays
x = np.array([[-1, -0.8], [1, -0.1], [0., 0.5]])
y = np.array([0.0, 0.0])
z = np.array([0.0, 0.0])
ipv.figure()
s = ipv.scatter(x, y, z, marker='sphere', size=10)
ipv.xyzlim(-1, 1)
ipv.animation_control(s) # shows controls for animation controls
ipv.show()

In [5]:
import numpy as np
from numpy import pi, sin, cos, radians
import ipyvolume as ipv
import ipywidgets as widgets

def transform_xyz(theta, phi, x, y, z):
    x, y, z = [np.asarray(v) for v in (x, y, z)]
    shape = x.shape
    points = np.matrix([x.flatten(), y.flatten(), z.flatten()])
    Ry_theta = [[+cos(theta), 0, +sin(theta)],
                [0, 1, 0],
                [-sin(theta), 0, +cos(theta)]]
    Rz_phi = [[+cos(phi), -sin(phi), 0],
              [+sin(phi), +cos(phi), 0],
              [0, 0, 1]]
    points = np.matrix(Rz_phi)*np.matrix(Ry_theta)*points
    x, y, z = [np.array(v).reshape(shape) for v in points]
    return x, y, z

def torus(R=100, r=10, Rsteps=100, rsteps=100):
    u = np.linspace(0, 2*pi, rsteps) # tube
    v = np.linspace(0, 2*pi, Rsteps) # ring
    U, V = np.meshgrid(u, v)
    x = (R + r*cos(U))*cos(V)
    y = (R + r*cos(U))*sin(V)
    z = r*sin(U)
    return x, y, z

class Scene:
    def __init__(self, view=(0, 0, 0), hkl=(1,1,1)):
        self.view = view
        self.figure = ipv.figure()
        R = 100.
        r = R/30.
        ipv.xlim(-R, R)
        ipv.ylim(-R, R)
        ipv.zlim(-R, R)
        ipv.style.box_off()
        self.torus_xyz = torus(R=R, r=R/10)
        self.torus = ipv.plot_surface(*self.torus_xyz)
        ipv.show()
        self.ui = widgets.interact(self.update, theta=(-90.,90.), phi=(-180., 180.))
        #self.ui.widget.children[0].continuous_update = False
        #self.ui.widget.children[1].continuous_update = False

    def update(self, theta, phi):
        print("update with", theta, phi)
        x, y, z = transform_xyz(radians(theta), radians(phi), *self.torus_xyz)
        self.torus.x = x.flatten()
        self.torus.y = y.flatten()
        self.torus.z = z.flatten()

scene = Scene()

interactive(children=(FloatSlider(value=0.0, description='theta', max=90.0, min=-90.0), FloatSlider(value=0.0,…

In [6]:
import numpy as np
from numpy import pi, sin, cos, radians
import ipyvolume as ipv
import ipywidgets as widgets

# create 2d grids: x, y, and r
u = np.linspace(-10, 10, 25)
x, y = np.meshgrid(u, u)
r = np.sqrt(x**2+y**2)
print("x,y and z are of shape", x.shape)
# and turn them into 1d
x = x.flatten()
y = y.flatten()
r = r.flatten()
print("and flattened of shape", x.shape)

# create a sequence of 15 time elements
time = np.linspace(0, np.pi*2, 15)
z = np.array([(np.cos(r + t) * np.exp(-r/5)) for t in time])
print("z is of shape", z.shape)

# draw the scatter plot, and add controls with animate_glyphs
ipv.figure()
s = ipv.scatter(x, z, y, marker="sphere")
ipv.animation_control(s, interval=200)
ipv.ylim(-3,3)
ipv.show()

# Now also include, color, which containts rgb values
color = np.array([[np.cos(r + t), 1-np.abs(z[i]), 0.1+z[i]*0] for i, t in enumerate(time)])
size = (z+1)
print("color is of shape", color.shape)

color = np.transpose(color, (0, 2, 1)) # flip the last axes

ipv.figure()
s = ipv.scatter(x, z, y, color=color, size=size, marker="sphere")
ipv.animation_control(s, interval=200)
ipv.ylim(-3,3)
ipv.show()

x,y and z are of shape (25, 25)
and flattened of shape (625,)
z is of shape (15, 625)


color is of shape (15, 3, 625)


In [ ]:
import numpy as np
from numpy import pi, sin, cos, radians
import ipyvolume as ipv
import ipywidgets as widgets


def set_view(figure, framenr, fraction):
    ipv.view(fraction*360, (fraction - 0.5) * 180, distance=2 + fraction*2)
    s.size = size * (2+0.5*np.sin(fraction * 6 * np.pi))
ipv.movie('wave.gif', set_view, fps=20, frames=40)

# include the gif with base64 encoding
import IPython.display
import base64
with open('wave.gif', 'rb') as gif:
    url = b"data:image/gif;base64," +base64.b64encode(gif.read())
IPython.display.Image(url=url.decode('ascii'))

Output()

In [7]:
import numpy as np
from numpy import pi, sin, cos, radians
import ipyvolume as ipv
import ipywidgets as widgets
import ipyvolume.datasets

stream = ipyvolume.datasets.animated_stream.fetch()
print("shape of steam data", stream.data.shape) # first dimension contains x, y, z, vx, vy, vz, then t

fig = ipv.figure()
# instead of doing x=stream.data[0], y=stream.data[1], ... vz=stream.data[5], use *stream.data
# limit to 50 timesteps to avoid having a huge notebook
q = ipv.quiver(*stream.data[:,0:50,:200], color="red", size=7)
ipv.style.use("dark") # looks better
ipv.animation_control(q, interval=200)
ipv.show()

ERROR! Session/line number was not unique in database. History logging moved to new session 17
shape of steam data (6, 200, 1250)


In [ ]:
import numpy as np
import vaex
from numpy import pi, sin, cos, radians
import ipyvolume as ipv
import ipywidgets as widgets
import ipyvolume.datasets

ds = vaex.example()
N = 2000 # for performance reasons we only do a subset
x, y, z, vx, vy, vz, Lz, E = [ds.columns[k][:N] for k in "x y z vx vy vz Lz E".split()]

import bqplot.pyplot as plt

plt.figure(1, title="E Lz space")
scatter = plt.scatter(Lz, E,
                selected_style={'opacity': 0.2, 'size':1, 'stroke': 'red'},
                unselected_style={'opacity': 0.2, 'size':1, 'stroke': 'blue'},
                default_size=1,
               )
plt.brush_selector()
plt.show()

import ipyvolume.pylab as p3

p3.clear()
quiver = p3.quiver(x, y, z, vx, vy, vz, size=2, size_selected=5, color_selected="blue")
p3.show()

from ipywidgets import jslink, VBox

jslink((scatter, 'selected'), (quiver, 'selected'))

hbox = VBox([p3.current.container, plt.figure(1)])
hbox

In [ ]:
import k3d
import numpy as np

lines = np.load('vertices.npy')
lines_attributes = np.load('attributes.npy')

plot = k3d.plot()

for l, a in zip(lines, lines_attributes):
    plot += k3d.line(l, attribute=a, width=0.0001,
                     color_map=k3d.matplotlib_color_maps.Inferno, color_range=[0,0.5], shader='mesh',
                     compression_level=9)
plot.display()

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')
theta = np.linspace(-4 * np.pi, 4 * np.pi, 100)
z = np.linspace(-2, 2, 100)
r = z**2 + 1
x = r * np.sin(theta)
y = r * np.cos(theta)
ax.plot(x, y, z, label='parametric curve')
ax.legend()

plt.show()